In [1]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
import fasttext
import fasttext.util
ft = fasttext.load_model('cc.zh.300.bin')
ft.get_dimension()

300

In [2]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['punc_rumor'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['punc_truth'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)


In [3]:
# jieba分词
import jieba
# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
 
seg_list = []
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    seg_list.append( ' '.join(words))
 
df['seg_text'] = seg_list

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/jieba.cache
Loading model cost 0.358 seconds.
Prefix dict has been built successfully.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize,ngram_range=(1,2))
tfidf_docs = tfidf_model.fit_transform(raw_documents=df.seg_text).toarray()#这句话计算了tfidf
words = tfidf_model.get_feature_names_out()
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(tfidf_docs)
cos_avg = sum([sum(row) for row in cosine_similarities])/len(cosine_similarities)
cos_sim = [sum(row)/cos_avg for row in cosine_similarities]
df['cos_sim'] = cos_sim
def get_word_vector(word):
    return ft.get_word_vector(word)
for i,word in enumerate(words):
    if i == 0:
        word_vectors = get_word_vector(word)
    else:
        word_vectors = np.vstack((word_vectors,get_word_vector(word)))
word_dict = dict(zip(words,word_vectors))
np.savez('../dataset/word_dict.npz',word_dict = word_dict)



/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:
#snowNLP情感分析
from snownlp import SnowNLP
sentimentslist = []
for i in (df['text']):
    a1 = SnowNLP(i)
    a2 = a1.sentiments
    sentimentslist.append(a2)
avg_len = sum([len(x) for x in df['text']])/len(df['text'])
df['length'] = [len(x)/avg_len for x in df['text']]
df['sentiments'] = sentimentslist

In [6]:
from sklearn.decomposition import PCA
k=min(len(tfidf_model.vocabulary_),len(df))
k=100
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
pca_topic_vectors.round(3).head(6)



,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
rumor0!,0.042,-0.031,0.034,0.107,0.134,0.056,-0.100,0.165,0.114,-0.088,...,0.078,0.015,-0.022,0.012,-0.005,0.025,0.008,-0.005,-0.029,-0.033
rumor1!,-0.003,-0.004,-0.011,-0.044,-0.001,-0.009,-0.030,0.000,-0.044,0.030,...,0.015,0.026,-0.007,-0.002,-0.004,-0.007,-0.003,0.006,0.006,0.010
rumor2,-0.007,-0.021,-0.019,-0.058,-0.003,0.018,-0.040,-0.013,-0.054,0.006,...,0.007,0.047,-0.002,0.017,-0.007,-0.010,-0.027,-0.012,-0.031,-0.020
rumor3,0.028,-0.028,-0.051,0.060,0.027,-0.027,-0.004,-0.039,0.022,0.005,...,-0.023,-0.008,-0.030,0.010,-0.023,0.022,-0.057,-0.004,-0.018,-0.050
rumor4!,0.006,-0.018,-0.016,0.015,-0.008,0.090,0.027,0.031,0.019,0.002,...,0.000,-0.018,-0.004,0.011,0.018,-0.015,-0.010,-0.015,0.021,-0.002
rumor5,0.006,-0.011,-0.026,-0.016,0.007,0.018,-0.008,-0.020,0.008,0.012,...,-0.001,-0.010,0.019,0.027,0.006,-0.007,-0.010,0.019,0.006,-0.033


In [7]:
df['pca_topic_vectors'] = pca_topic_vectors.values.tolist()

In [8]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia = LDiA(n_components=k, learning_method='batch')
ldia = ldia.fit(bow_docs)  # <1>
ldia.components_.shape
ldia32_topic_vectors = ldia.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(ldia.components_.shape[0])]
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors,index=index, columns=columns32)
ldia32_topic_vectors.round(2).head()

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
rumor0!,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor1!,0.0,0.0,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor2,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor3,0.0,0.0,0.00,0.0,0.8,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rumor4!,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df['ldia_topic_vectors'] = ldia32_topic_vectors.values.tolist()

In [10]:
df.to_csv('../dataset/data.csv',encoding='utf-8')
data = pd.read_csv('../dataset/data.csv',encoding='utf-8',index_col=0)